#**Binary Classification Using Deep Neural Network For Predicting Heart Attacks**


>Submitted to: **Dr. Preety Singh**






### **1. Import Libraries**


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import seaborn as sns

### **2. Loading and Preprocessing the Dataset**

In [ ]:
def preprocess_data(data):
    # Drop rows with missing values
    data = data.dropna()

    # Convert 'Result' to binary
    data['Result'] = data['Result'].map({'negative': 0, 'positive': 1})

    # Standardization to Normal Distribution with mean 0 and variance 1
    scaler = StandardScaler()
    features = ['Age', 'Gender', 'Heart rate', 'Systolic blood pressure',
                'Diastolic blood pressure', 'Blood sugar', 'CK-MB', 'Troponin']

    data[features] = scaler.fit_transform(data[features])

    # Add polynomial features for important variables
    data['CK_MB_squared'] = data['CK-MB'] ** 2
    data['Troponin_squared'] = data['Troponin'] ** 2
    data['Age_squared'] = data['Age'] ** 2

    # Add interaction terms
    data['CK_MB_Troponin'] = data['CK-MB'] * data['Troponin']
    data['Age_BloodSugar'] = data['Age'] * data['Blood sugar']

    features = features + ['CK_MB_squared', 'Troponin_squared', 'Age_squared',
                          'CK_MB_Troponin', 'Age_BloodSugar']

    X = data[features].values.T
    Y = data['Result'].values.reshape(1, -1)

    return data, X, Y

### **3: Neural Network Initialization**


In [ ]:
def initialize_parameters(layer_dims):
    parameters = {}
    L = len(layer_dims)

    for l in range(1, L):
        parameters[f'W{l}'] = np.random.randn(layer_dims[l], layer_dims[l-1]) * np.sqrt(2./layer_dims[l-1]) #He Normal Initialization because of ReLU activation used in hidden layers.
        parameters[f'b{l}'] = np.zeros((layer_dims[l], 1))

    return parameters


### **4: Forward Propagation**



In [ ]:
def forward_propagation(X, parameters, keep_prob=0.8):
    caches = []
    A = X
    L = len(parameters) // 2

    for l in range(1, L):
        A_prev = A
        W = parameters[f'W{l}']
        b = parameters[f'b{l}']

        Z = np.dot(W, A_prev) + b
        A = np.maximum(0, Z)  # ReLU

        # Apply dropout
        D = np.random.rand(A.shape[0], A.shape[1]) < keep_prob
        A = (A * D) / keep_prob #dividing by keep_prob to ensure that the expected output activations remain the same. Since number of neurons get reduced, we need to scale up the output accordingly.

        cache = ((A_prev, W, b), Z, D)
        caches.append(cache)

    # Output layer with sigmoid
    WL = parameters[f'W{L}']
    bL = parameters[f'b{L}']
    ZL = np.dot(WL, A) + bL
    AL = 1 / (1 + np.exp(-ZL))

    caches.append(((A, WL, bL), ZL))

    return AL, caches

### **5: Cost Computation**


In [ ]:
def compute_cost(AL, Y, parameters, lambd=0.01):
    m = Y.shape[1]

    # Cross-entropy loss
    cross_entropy_cost = -1/m * np.sum(Y * np.log(AL + 1e-8) + (1-Y) * np.log(1-AL + 1e-8)) #AL is y hat. Added 1e-8 to avoid computing log of 0.

    # L2 regularization
    L2_regularization_cost = 0
    L = len(parameters) // 2 #Because the parameters data structure contains both weights and biases for each layer and we need only weights, therefore we divide by 2.
    for l in range(L):
        L2_regularization_cost += np.sum(np.square(parameters[f'W{l+1}']))
    L2_regularization_cost *= lambd/(2*m)

    return cross_entropy_cost + L2_regularization_cost

### **6: Backward Propagation**


In [ ]:
def backward_propagation(AL, Y, caches, keep_prob=0.8, lambd=0.01):
    grads = {}
    L = len(caches)
    m = AL.shape[1]
    Y = Y.reshape(AL.shape)

    # derivative of binary cross-entropy:
    dAL = -(np.divide(Y, AL + 1e-8) - np.divide(1-Y, 1-AL + 1e-8))
    current_cache = caches[L-1]

    dA_prev_temp, dW_temp, db_temp = linear_activation_backward(
        dAL, current_cache, activation="sigmoid", lambd=lambd)
    grads[f"dW{L}"] = dW_temp
    grads[f"db{L}"] = db_temp

    # Hidden layers
    for l in reversed(range(L-1)): #L-1 because the input layer doesn't count in hidden layers
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(
            dA_prev_temp, current_cache, activation="relu", lambd=lambd, keep_prob=keep_prob)
        grads[f"dW{l+1}"] = dW_temp
        grads[f"db{l+1}"] = db_temp

    return grads

#Funciton to perform backward propagation for single layer in the neural network:
def linear_activation_backward(dA, cache, activation, lambd=0.01, keep_prob=0.8):
    linear_cache, activation_cache, D = cache if len(cache) == 3 else (*cache, None)
    A_prev, W, b = linear_cache
    m = A_prev.shape[1]

    if activation == "relu":
        dZ = dA * (activation_cache > 0)
        if D is not None:  # Apply dropout
            dZ = dZ * D / keep_prob
    elif activation == "sigmoid":
        s = 1/(1+np.exp(-activation_cache))
        dZ = dA * s * (1-s)

    dW = 1./m * np.dot(dZ, A_prev.T) + (lambd/m) * W  #Derivative changed due to L2 Regularization
    db = 1./m * np.sum(dZ, axis=1, keepdims=True)
    dA_prev = np.dot(W.T, dZ)

    return dA_prev, dW, db

### **7: Update Parameters**


In [ ]:
def update_parameters(parameters, grads, learning_rate):
    L = len(parameters) // 2

    for l in range(L):
        parameters[f"W{l+1}"] -= learning_rate * grads[f"dW{l+1}"]
        parameters[f"b{l+1}"] -= learning_rate * grads[f"db{l+1}"]

    return parameters

### **8: Model Definition**


In [ ]:
def model(X, Y, layers_dims, learning_rate=0.01, num_epochs=3000, batch_size=32,
          keep_prob=0.8, lambd=0.01, print_cost=True):
    costs = []

    # Initialize parameters
    parameters = initialize_parameters(layers_dims)

    # Implement mini-batch gradient descent
    num_complete_batches = X.shape[1] // batch_size

    for i in range(num_epochs):
        epoch_cost = 0

        # Shuffle the data
        permutation = np.random.permutation(X.shape[1])
        X_shuffled = X[:, permutation]
        Y_shuffled = Y[:, permutation]

        for k in range(num_complete_batches):
            # Get mini-batch
            start_idx = k * batch_size
            end_idx = (k + 1) * batch_size
            X_batch = X_shuffled[:, start_idx:end_idx]
            Y_batch = Y_shuffled[:, start_idx:end_idx]

            AL, caches = forward_propagation(X_batch, parameters, keep_prob)
            cost = compute_cost(AL, Y_batch, parameters, lambd)
            grads = backward_propagation(AL, Y_batch, caches, keep_prob, lambd)
            parameters = update_parameters(parameters, grads, learning_rate)

            epoch_cost += cost

        # Print the cost every 100 epochs
        if print_cost and i % 100 == 0:
            print(f"Cost after epoch {i}: {epoch_cost}")
        if print_cost :
            costs.append(epoch_cost)

    return parameters, costs

### **9: Load and Preprocess Data**



In [ ]:
data = pd.read_csv('Medicaldataset.csv')
data, X, Y = preprocess_data(data)

### **10: Plot Functions**





In [ ]:
def plot_scatterplots(df):
    # List of columns to plot
    numerical_columns = ['Age', 'Heart rate', 'Systolic blood pressure', 'Diastolic blood pressure', 'Blood sugar', 'CK-MB', 'Troponin']

    # Color mapping for the 'Result' column: positive = red, negative = green
    color_map = {'positive': 'red', 'negative': 'green'}
    colors = df['Result'].map(color_map)

    # Create scatter plots for each pair of columns
    num_cols = len(numerical_columns)
    plt.figure(figsize=(16, 12))

    plot_number = 1
    for i in range(num_cols):
        for j in range(i + 1, num_cols):
            plt.subplot(num_cols - 1, num_cols - 1, plot_number)
            plt.scatter(df[numerical_columns[i]], df[numerical_columns[j]], c=colors)
            plt.xlabel(numerical_columns[i])
            plt.ylabel(numerical_columns[j])
            plot_number += 1

    plt.tight_layout()
    plt.show()

def plot_correlation_matrix(df):
    # List of numerical columns
    numerical_columns = ['Age', 'Heart rate', 'Systolic blood pressure', 'Diastolic blood pressure', 'Blood sugar', 'CK-MB', 'Troponin']
    corr_matrix = df[numerical_columns].corr()
    # Use only upper triangle
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
    plt.figure(figsize=(10, 8))
    # Create a heatmap using seaborn
    sns.heatmap(corr_matrix, annot=True, mask=mask, cmap="coolwarm", vmin=-1, vmax=1, square=True, linewidths=0.5, cbar_kws={"shrink": .8})
    plt.title("Correlation Matrix of Medical Data", fontsize=16)
    plt.show()

# Generate the Plots
untouchedData = pd.read_csv('Medicaldataset.csv')
plot_scatterplots(untouchedData)
print(' ')
plot_correlation_matrix(untouchedData)



### **11: Make Predictions and Evaluate**


In [ ]:
# Split data
X_train, X_test, Y_train, Y_test = train_test_split(X.T, Y.T, test_size=0.2, random_state=42)
X_train, X_test = X_train.T, X_test.T
Y_train, Y_test = Y_train.T, Y_test.T

# Define layer dimensions
layers_dims = [X_train.shape[0], 15, 8, 1]  # 2 hidden layers

# Train the model
parameters, costs = model(X_train, Y_train, layers_dims,
                         learning_rate=0.01,
                         num_epochs=3000,
                         batch_size=32,
                         keep_prob=0.8,
                         lambd=0.01)

def plot_costs(costs):
    epochs = range(len(costs))

    # Plotting the cost vs. epochs
    print('  ')
    plt.plot(epochs, costs, label='Cost')
    plt.xlabel('Epochs')
    plt.ylabel('Cost')
    plt.title('Cost vs. Epochs')
    plt.legend()
    plt.show()

plot_costs(costs)


# Generate predictions for the test set
Y_test_predicted, _ = forward_propagation(X_test, parameters, keep_prob=1.0)  # No dropout during testing
Y_test_predicted = (Y_test_predicted > 0.5).astype(int)  # Convert probabilities to binary predictions

# Calculate the confusion matrix
cm = confusion_matrix(Y_test.flatten(), Y_test_predicted.flatten())

# Display the confusion matrix
plt.figure(figsize=(8, 6))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Negative', 'Positive'])
disp.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.show()
# Calculate accuracy
def calculate_accuracy(X, Y, parameters):
    AL, _ = forward_propagation(X, parameters, keep_prob=1.0)  # No dropout during testing
    predictions = (AL > 0.5).astype(int)
    accuracy = np.mean(predictions == Y) * 100
    return accuracy

# Print final accuracies
train_accuracy = calculate_accuracy(X_train, Y_train, parameters)
test_accuracy = calculate_accuracy(X_test, Y_test, parameters)
print(f"Final Training Accuracy: {train_accuracy:.2f}%")
print(f"Final Test Accuracy: {test_accuracy:.2f}%")